In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
@tool
def web_loader(url: str) -> str:
    """抓取url对应网页的内容"""
    loader = WebBaseLoader(url)
    docs = loader.load()
    return docs[0].page_content

In [3]:
llm = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=4096)
tools = [web_loader]
llm_with_tools = llm.bind_tools(tools)


In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个擅长对长文做总结的智能助手，可以精确提炼出长文中的要点。注意，请使用markdown格式返回结果。",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [5]:
prompt

ChatPromptTemplate(input_variables=['input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': [], 'agent_scratchpad': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='你是一个擅长对长文做总结的智能助手，可以精确提炼出长文中的要点。注意，请使用markdown格式返回结果。')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate

In [6]:
agent = create_tool_calling_agent(llm_with_tools, tools, prompt)

In [7]:

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [8]:
res = agent_executor.invoke(
    {
        "input": "这个链接讲了什么内容？ https://blog.csdn.net/xindoo/article/details/138356308?spm=1001.2014.3001.5501"
    }
)

In [9]:
res

{'input': '这个链接讲了什么内容？ https://blog.csdn.net/xindoo/article/details/138356308?spm=1001.2014.3001.5501',
 'output': '# 文章总结\n\n- **标题**: 推荐一个好用的命令行工具ShellGPT\n- **作者**: xindoo\n- **发布日期**: 2024-04-30\n- **阅读量**: 595，收藏 21，点赞 15\n\n## 内容概要\n文章介绍了一个开源软件ShellGPT，可以帮助用户快速生成shell命令、代码片段和文档，无需依赖外部资源。主要内容包括：\n1. **配置安装**: 支持Linux、macOS、Windows系统，通过pip安装，依赖于外部大语言模型。\n2. **常用功能**:\n    - 聊天功能：使用ShellGPT可以进行基础的单轮和多轮聊天。\n    - 写命令并执行：ShellGPT可以帮助用户生成并执行命令。\n3. **高级功能**:\n    - 函数调用：ShellGPT支持LLM执行系统中的函数，提供更复杂的功能。\n    - 角色管理：用户可以创建自定义角色，执行特定功能。\n4. **总结**: 文章介绍了ShellGPT的基本功能和高级功能，鼓励读者自行探索更多细节。\n\n作者提到，由于篇幅限制，只介绍了部分内容，建议通过查看完整参数信息和使用`sgpt --help`来了解更多功能。文章结尾作者分享了一些相关推荐文章和专栏。\n\n详细内容请访问[原文链接](https://blog.csdn.net/xindoo/article/details/138356308)。'}